In [1]:
%load_ext autoreload

In [49]:
import pandas as pd
import geopandas as gpd
import datetime as dt

AttributeError: 'LGEOS360' object has no attribute 'GEOSMinimumClearance'

Read in downloaded CSV file:

In [3]:
chi_311 = pd.read_csv('../raw_data/chicago_311_requests.csv')

/Users/michaelfeldman/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (11,12,18,19,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
chi_311.shape

(4080622, 37)

Filter dataframe to exclude info-only calls:

In [5]:
chi_311_filtered = chi_311[chi_311['SR_TYPE'] != '311 INFORMATION ONLY CALL']

In [6]:
chi_311_filtered.shape

(2702596, 37)

Filter for columns needed:

In [7]:
chi_311_filtered.columns

Index(['SR_NUMBER', 'SR_TYPE', 'SR_SHORT_CODE', 'OWNER_DEPARTMENT', 'STATUS',
       'CREATED_DATE', 'LAST_MODIFIED_DATE', 'CLOSED_DATE', 'STREET_ADDRESS',
       'CITY', 'STATE', 'ZIP_CODE', 'STREET_NUMBER', 'STREET_DIRECTION',
       'STREET_NAME', 'STREET_TYPE', 'DUPLICATE', 'LEGACY_RECORD',
       'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER', 'COMMUNITY_AREA', 'WARD',
       'ELECTRICAL_DISTRICT', 'ELECTRICITY_GRID', 'POLICE_SECTOR',
       'POLICE_DISTRICT', 'POLICE_BEAT', 'PRECINCT',
       'SANITATION_DIVISION_DAYS', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
       'CREATED_MONTH', 'X_COORDINATE', 'Y_COORDINATE', 'LATITUDE',
       'LONGITUDE', 'LOCATION'],
      dtype='object')

In [8]:
chi_311_filtered = chi_311_filtered[['SR_NUMBER', 'SR_TYPE', 'OWNER_DEPARTMENT',
                                     'STATUS', 'CREATED_DATE', 'CLOSED_DATE', 'DUPLICATE',
                                     'LEGACY_RECORD', 'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER',
                                     'COMMUNITY_AREA', 'WARD', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
                                     'CREATED_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION']]

In [9]:
chi_311_filtered.shape

(2702596, 18)

Filter out legacy records:

In [10]:
legacy = chi_311_filtered[chi_311_filtered['LEGACY_RECORD'] == True]

In [11]:
max(legacy['CREATED_DATE'])

'12/18/2018 12:47:28 AM'

In [12]:
legacy.shape

(421791, 18)

In [13]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['LEGACY_RECORD'] == False]
chi_311_filtered = chi_311_filtered.drop(columns=['LEGACY_RECORD', 'LEGACY_SR_NUMBER'])

In [14]:
chi_311_filtered.shape

(2280805, 16)

Check which rows have no location and remove them:

In [15]:
no_loc = chi_311_filtered[chi_311_filtered['LOCATION'].isna()]

In [16]:
no_loc.shape

(25165, 16)

In [17]:
# Make sure records with no location also have no latitude
no_lat = chi_311_filtered[chi_311_filtered['LATITUDE'].isna()]

In [18]:
no_lat.shape

(25165, 16)

In [19]:
# Check which request types have no location (looks to be mostly noise complaints)
no_loc['SR_TYPE'].value_counts()

Aircraft Noise Complaint                            17081
Graffiti Removal Request                             1470
Pothole in Street Complaint                          1115
Street Light Out Complaint                            885
Weed Removal Request                                  464
                                                    ...  
Tobacco - Sale to Minors Complaint                      1
Sidewalk Cafe Complaint                                 1
Recycling Inspection Request                            1
Porch Inspection Request                                1
Licensed Pharmaceutical Representative Complaint        1
Name: SR_TYPE, Length: 84, dtype: int64

In [20]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['LOCATION'].notna()]

In [21]:
chi_311_filtered.shape

(2255640, 16)

Filter out rows with no community area or ward

In [22]:
chi_311_filtered[chi_311_filtered['COMMUNITY_AREA'].isna()].shape

(1787, 16)

In [23]:
chi_311_filtered[chi_311_filtered['WARD'].isna()].shape

(1607, 16)

In [24]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['COMMUNITY_AREA'].notna() &
                                    chi_311_filtered['WARD'].notna()]

In [25]:
chi_311_filtered.shape

(2253842, 16)

Check and remove duplicates:

In [26]:
dupes = chi_311_filtered[chi_311_filtered['DUPLICATE'] == True]

In [27]:
dupes[dupes['PARENT_SR_NUMBER'] == 'SR19-02206459']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,LATITUDE,LONGITUDE,LOCATION
412,SR19-02557557,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/23/2019 06:13:16 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,18,2,9,41.879477,-87.763706,"(41.879477495788485, -87.76370588423917)"
674210,SR19-02412950,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/07/2019 06:59:08 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,6,7,9,41.879324,-87.761916,"(41.87932434546784, -87.76191601010542)"
766337,SR19-02786257,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,10/22/2019 02:16:57 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,14,3,10,41.879268,-87.762142,"(41.879267695685606, -87.76214183334518)"
2266793,SR19-01819033,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,06/28/2019 01:58:34 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,13,6,6,41.879324,-87.761916,"(41.87932434546784, -87.76191601010542)"
2266847,SR19-02412742,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/07/2019 05:11:59 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,5,7,9,41.879324,-87.761916,"(41.87932434546784, -87.76191601010542)"
2266848,SR19-02438203,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/10/2019 10:07:29 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,10,3,9,41.879049,-87.764287,"(41.87904861695672, -87.76428729934396)"
2266849,SR19-02438308,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/10/2019 10:11:17 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,10,3,9,41.879060,-87.764026,"(41.87905954923467, -87.76402622454499)"
2266867,SR19-02469775,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/12/2019 05:43:36 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,17,5,9,41.879324,-87.761916,"(41.87932434546784, -87.76191601010542)"
2266953,SR19-03092189,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,12/06/2019 10:44:39 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,22,6,12,41.879049,-87.764287,"(41.87904861695672, -87.76428729934396)"


In [28]:
dupes[dupes['SR_NUMBER'] == 'SR19-02206459']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,LATITUDE,LONGITUDE,LOCATION


In [29]:
# Looks if we remove rows where duplicate == 'True', then we don't delete the original parent request
chi_311_filtered[chi_311_filtered['SR_NUMBER'] == 'SR19-02206459']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,LATITUDE,LONGITUDE,LOCATION
761114,SR19-02206459,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,08/14/2019 03:40:48 PM,12/20/2019 03:19:51 PM,False,NaN,25.0,29.0,15,4,8,41.87938,-87.761907,"(41.87937977042881, -87.76190680466857)"


In [30]:
dupes[dupes['PARENT_SR_NUMBER'] == 'SR19-01473136']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,LATITUDE,LONGITUDE,LOCATION
40,SR19-01473141,Traffic Signal Out Complaint,CDOT - Department of Transportation,Open,05/11/2019 04:27:11 PM,NaN,True,SR19-01473136,55.0,10.0,16,7,5,41.680867,-87.540145,"(41.68086671767255, -87.54014478630506)"


In [31]:
chi_311_filtered[chi_311_filtered['SR_NUMBER'] == 'SR19-01473136']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,LATITUDE,LONGITUDE,LOCATION
3809668,SR19-01473136,Traffic Signal Out Complaint,CDOT - Department of Transportation,Open,05/11/2019 04:26:20 PM,NaN,False,NaN,55.0,10.0,16,7,5,41.680874,-87.540219,"(41.6808740411203, -87.54021864373331)"


In [32]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['DUPLICATE'] == False]
chi_311_filtered = chi_311_filtered.drop(columns=['DUPLICATE', 'PARENT_SR_NUMBER'])

In [33]:
chi_311_filtered.shape

(1991673, 14)

Filtered out complaints resolved in very short period of time -
NEED TO DECIDE WHICH COMPLAINTS TO FILTER OUT!!

In [34]:
test = chi_311_filtered.copy()

In [35]:
chi_311_filtered['CREATED_DATE'] = pd.to_datetime(chi_311_filtered['CREATED_DATE'],
                                                  format='%m/%d/%Y %I:%M:%S %p')
chi_311_filtered['CLOSED_DATE'] = pd.to_datetime(chi_311_filtered['CLOSED_DATE'],
                                                format='%m/%d/%Y %I:%M:%S %p')

In [36]:
chi_311_filtered['time_to_close'] = chi_311_filtered['CLOSED_DATE'] - \
                                        chi_311_filtered['CREATED_DATE']

In [37]:
chi_311_filtered['time_to_close_sec'] = chi_311_filtered['time_to_close'].dt.total_seconds()

In [38]:
# Requests resolved in 0 seconds
chi_311_filtered[chi_311_filtered['time_to_close_sec'] == 0].shape

(402913, 16)

In [39]:
# Requests resolved in less than 1 minute
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 60].shape

(770805, 16)

In [40]:
# Requests resolved in less than 10 minute
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 600].shape

(843289, 16)

In [41]:
# Requests resolved in less than 1 day
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 86400].shape

(1060276, 16)

In [42]:
# Types of requests resolved in less than 10 minutes
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 600]['SR_TYPE'].value_counts()

Aircraft Noise Complaint                                 584578
Weed Removal Request                                      95752
Graffiti Removal Request                                  81014
Sign Repair Request - All Other Signs                     38539
Pothole in Street Complaint                                8193
                                                          ...  
Bungalow/Vintage Home Information Request                     1
Protected Bike Lane - Debris Removal                          1
Clean and Green Program Request                               1
Dead Bird                                                     1
Snow Removal - Protected Bike Lane or Bridge Sidewalk         1
Name: SR_TYPE, Length: 77, dtype: int64

In [43]:
# Filter out requests resolved in less than a minute
chi_311_filtered = chi_311_filtered[chi_311_filtered['time_to_close_sec'] >= 60]

In [50]:
chi_311_filtered.shape

(1131941, 16)

In [46]:
chi_311_filtered.columns

Index(['SR_NUMBER', 'SR_TYPE', 'OWNER_DEPARTMENT', 'STATUS', 'CREATED_DATE',
       'CLOSED_DATE', 'COMMUNITY_AREA', 'WARD', 'CREATED_HOUR',
       'CREATED_DAY_OF_WEEK', 'CREATED_MONTH', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'time_to_close', 'time_to_close_sec'],
      dtype='object')

Pickle filtered file:

In [51]:
chi_311_filtered.to_pickle("../pickle_files/chi_311.pkl")